In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import transforms3d as t3d
import datetime
import torch
from torch.nn import functional as F

import sys
sys.path.append("../")
import curvvae_lib.train.predictive_passthrough_trainer as ppttrainer
import curvvae_lib.architecture.passthrough_vae as ptvae
import curvvae_lib.architecture.save_model as sm
import curvvae_lib.architecture.load_model as lm

Matplotlib created a temporary config/cache directory at /tmp/matplotlib-am7iyckj because the default path (/home/tsr42/.cache/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


In [2]:
foodname = "strawberry"
foldername = f"fork_trajectory_{foodname}"
savefilename = f"{foodname}_clean_pickups"

In [3]:
train = []
training_ts = np.linspace(0,1,64)
attempt = 1
while True:
    try:
        raw_vals = np.load(f"{savefilename}/pickup_attempt{attempt}.npy")
        train.append(raw_vals.T.flatten())
    except:
        print(f"We found {attempt-1} pickup attempts")
        break
    attempt += 1

train = np.array(train).reshape(-1,7,64)
all_points = train[:,:,:]

time_shape = list(all_points.shape)
time_shape[1] = 1
# why be smart when you can be dumb
t = np.ones(time_shape)
for i in range(time_shape[2]):
    t[:,:,i] = t[:,:,i] * i / (time_shape[2] + 0.0)

all_points = np.concatenate((t, all_points), axis=1)
all_points = all_points.transpose(0,2,1)

# See http://localhost:8889/notebooks/scratchwork/2021-09-17%20Rotation%20Scaling.ipynb
# for why we want quaternion values to be multiplied by 0.16 when position values are in meters 
# (if the relevant distance scale of the fork is 0.08 meters, ie: 8cm).
# we additionally multiply everything by 1000 to reduce the likelihood of model collapse for reasonable beta values
rotation_error_scaling = 0.16
overall_scaling = 1000
ovs = overall_scaling
rts = rotation_error_scaling * overall_scaling
all_points = all_points * np.array((1,ovs,ovs,ovs,rts,rts,rts,rts))
print("shape before reducing dimension", all_points.shape)

We found 129 pickup attempts
shape before reducing dimension (129, 64, 8)


In [4]:
np.max(all_points[:,:,1:4])

41.56877331249602

In [5]:
def LoadDataBatch(all_points, batchsize, passthroughdim, predictive):
    """Sample Pair of Points from Trajectory"""
    # all_points should be of dimension: num_trajectories, numtimesteps, 1+spatialdims
    traj_ids = np.random.choice(all_points.shape[0], batchsize)
    t1_ids = np.random.choice(all_points.shape[1], batchsize)
    if predictive:
        t2_ids = np.random.choice(all_points.shape[1], batchsize)
    else:
        t2_ids = t1_ids
    return (torch.tensor(all_points[traj_ids, t1_ids,passthroughdim:], dtype=torch.float),
            torch.tensor(all_points[traj_ids, t1_ids,:passthroughdim], dtype=torch.float),
            torch.tensor(all_points[traj_ids, t2_ids,passthroughdim:], dtype=torch.float),
            torch.tensor(all_points[traj_ids, t2_ids,:passthroughdim], dtype=torch.float))

In [6]:
print(all_points.shape)

(129, 64, 8)


In [7]:
class Loader(object):
    def __init__(self, all_points, batchsize, passthroughdim, epochnumbatches, predictive):
        self.all_points = all_points
        self.batchsize = batchsize
        self.passthroughdim = passthroughdim
        self.epochnumbatches = epochnumbatches
        self.predictive = predictive
        
    def __iter__(self):
        self.n = 0
        return self
    
    def __next__(self):
        if self.n >= self.epochnumbatches:
            # https://docs.python.org/3/library/exceptions.html#StopIteration
            raise StopIteration
        self.n += 1
        return LoadDataBatch(self.all_points, self.batchsize, self.passthroughdim, self.predictive)
    

train_loader = Loader(all_points, 256, 1,10000,predictive=False)
    

In [8]:

train_loader = Loader(all_points, 256, 1,100,predictive=True)
for allxs, allts, testx2, testt2 in train_loader:
    print(allts[:1],testt2[:1])
    break

tensor([[0.0469]]) tensor([[0.4062]])


In [9]:
def make_vae(latent_dim):
    input_dim = 7 
    passthrough_dim = 1
    emb_layer_widths = [1000]
    recon_layer_widths = [1000]
    dtype = torch.float
    model = ptvae.FCPassthroughVAE(input_dim, passthrough_dim, latent_dim,
        emb_layer_widths, recon_layer_widths, dtype)
    return model
allxs, allts, t2s, x2s = LoadDataBatch(all_points, 3000, 1, predictive=False)

In [10]:
testname = f"model/{foodname}_clean_3lat/"

In [ ]:
all_exceptions = []
for curvreg in [100000,300000]:
    for beta in [1]:
        for latentdim in [3]:
            for epsilon_scale_start in [0.1]:
                try:
                    vae = make_vae(latent_dim=latentdim)
                    device = "cpu"
                    num_epochs = 1000
                    savedir  = f'{testname}lat{latentdim}_secreg0_curvreg{curvreg}_beta{beta}_{datetime.datetime.now().strftime("%Y%m%d-%H%M%S")}'
                    print(savedir)
                    trainer = ppttrainer.PPTVAETrainer(vae, train_loader, beta=beta, device=device,
                        log_dir=savedir, lr=0.0005, annealingBatches=0, record_loss_every=300, loss_func = "gaussian")
                    scheduler = torch.optim.lr_scheduler.ExponentialLR(trainer.optimizer, gamma=0.22)
                    epoch = 0
                    epsilon_scale = epsilon_scale_start
                    for _ in range(num_epochs*3):
                        epoch += 1
                        # had an off-by-one error here before
                        # update values BEFORE training 501st element
                        if epoch % num_epochs == 1 and epoch > 1: 
                            scheduler.step()
                            trainer.beta = trainer.beta * 1.0 #0.1
                            epsilon_scale = epsilon_scale * 0.8 #0.1
                        embst = trainer.train(second_deriv_regularizer=0, curvature_regularizer=curvreg, epsilon_scale = epsilon_scale, num_new_samp_points=128)
                        is_error = embst[0]
                        if is_error:
                            raise Exception


                        if epoch % num_epochs == 0:
                            modeled = vae.noiseless_autoencode(allxs, allts)[0].detach().cpu().numpy()
                            print(np.sum(np.square(modeled - allxs.detach().cpu().numpy())))
                            print('====> Epoch: {}'.format(epoch))
                            sm.save_fcpassthrough_vae(vae, savedir+f"_epoch{epoch}")
                    sm.save_fcpassthrough_vae(vae, savedir)
                    print(savedir)
                except e:
                    all_exceptions.append(embst)

model/strawberry_clean_3lat/lat3_secreg0_curvreg100000_beta1_20220126-151724
224390.14
====> Epoch: 1000
